In [9]:
import ccxt.pro as ccxtpro
import asyncio
import nest_asyncio
import pytz
from datetime import datetime, timedelta
import pandas as pd
# from binance_api import api_key, api_secret
from ccxt_lab import whale_1, api_key, api_secret

# Use nest_asyncio to allow async/await code to run in Jupyter Notebook
nest_asyncio.apply()
async def fetch_ohlcv_dataframe(exchange, symbol):
    try:
        # Replace 'YOUR_START_TIMESTAMP' and 'YOUR_END_TIMESTAMP' with the desired timestamps
        current_time_utc = datetime.utcnow() - timedelta(hours=1)

        # Format the current time as a string in the required format ('%Y-%m-%dT%H:%M:%SZ')
        start_time_str = current_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')

        # Parse the formatted string to create the start_timestamp
        start_timestamp = exchange.parse8601(start_time_str)

        # Fetch historical OHLCV data for the specified trading pair and timeframe
        ohlcv = await exchange.fetch_ohlcv(symbol, timeframe='1m', since=start_timestamp, limit=None)

        # Create a DataFrame from the fetched data
        df = pd.DataFrame(ohlcv, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
        df['Timestamp'] = df['Timestamp'] + timedelta(hours=8)
        df['Last5VolumeSum'] = df['Volume'].rolling(window=6, min_periods=1).sum() - df['Volume']

        return df

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

async def watch_ohlcv_and_print(exchange, symbol, whale):
    utc_plus_8 = pytz.timezone('Asia/Shanghai')
    previous_timestamp = None
    entry_time_list = []
    try:
        while True:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe='1m')
            
            timestamp_utc = datetime.fromtimestamp(ohlcv[-1][0] / 1000, tz=pytz.utc)
            timestamp_utc8 = timestamp_utc.astimezone(utc_plus_8)
            current_timestamp_str = timestamp_utc8.strftime('%Y-%m-%d %H:%M:%S')
            current_volume = ohlcv[-1][5]

            # Check if the timestamp has changed
            if current_timestamp_str != previous_timestamp:
                print(f"New timestamp: {current_timestamp_str}")
                df = await fetch_ohlcv_dataframe(exchange, symbol)
                previous_timestamp = current_timestamp_str

            # if df is not None and current_volume > df['Last5VolumeSum'].iloc[-1]:
            # test
            print(f"current volume: {ohlcv[-1][5]} ,test value: {df['Volume'].iloc[-2]}")

            # and not in this minute 
            if df is not None and current_volume > df['Volume'].iloc[-2] :
                # print(f"Last 5 volume sum: {df['Last5VolumeSum'].iloc[-1]}, Current volume: {current_volume}")

                if (whale.is_position_open() is False) and (current_timestamp_str not in entry_time_list):
                    try:
                        direction = 'buy' if ohlcv[-1][4] > ohlcv[-1][1] else 'sell'

                        place_order = whale.place_first_market_order(direction)
                        print(f"Place order: {place_order}")
                        entry_time_list.append(current_timestamp_str)
                        take_profit_order, stop_loss_order = whale.place_second_take_profit_and_stop_loss_orders()

                    except Exception as e:
                        print(f"An error occurred: {e}")

            

            print(f"Time: {current_timestamp_str}, Open: {ohlcv[-1][1]}, High: {ohlcv[-1][2]}, Low: {ohlcv[-1][3]}, Close: {ohlcv[-1][4]}, Volume: {ohlcv[-1][5]}, Last 5 volume sum: {df['Last5VolumeSum'].iloc[-1]:.2f}")


            # await asyncio.sleep(1)

    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

async def constantly_watch_position(exchange, symbol, whale):
    prev_position_exists = False  # Initialize prev_position_exists as False

    try:
        while True:
            # global position_exists
            positions = await exchange.fetch_positions([symbol])
            current_position_Amt_abs = abs(float(positions[0]['info']['positionAmt']))
            position_exists = (current_position_Amt_abs > 0 or current_position_Amt_abs < 0) 
             
            if position_exists and not prev_position_exists:
                print("Position exists from None")
                # Assuming 'test.close_position_and_orders()' is defined appropriately
            
            if not position_exists and prev_position_exists:
                print("No position anymore")
                whale.close_position_and_orders()
                # Assuming 'test.close_position_and_orders()' is defined appropriately

            prev_position_exists = position_exists
            # await asyncio.sleep(1)

    except KeyboardInterrupt:
        pass
    finally:
        await exchange.close()

# Replace 'exchange' and 'symbol' with your actual exchange object and symbol.
# You also need to define 'test.close_position_and_orders()' appropriately.

async def main():
    trading_symbol = 'ETH/USDT'

    exchange = ccxtpro.binance({
        'options': {
            'defaultType': 'future',  # Set the default market type to 'future'
            'newUpdates': True
        },
        'api_key': api_key,
        'secret': api_secret
    })
    # test = Whale(exchange, trading_symbol)
    exchange.set_sandbox_mode(True)
    tasks = [
        watch_ohlcv_and_print(exchange, trading_symbol, whale_1),
        constantly_watch_position(exchange, trading_symbol, whale_1)
    ]
    await asyncio.gather(*tasks)

# Run the async/await code in a Jupyter Notebook cell

api_key = api_key
api_secret = api_secret

asyncio.run(main())


New timestamp: 2023-10-06 20:27:00
current volume: 33.109 ,test value: 17.166
Place order: {'info': {'orderId': '1181968724', 'symbol': 'ETHUSDT', 'status': 'FILLED', 'clientOrderId': 'x-xcKtGhcu817b5d191973e9f64c6874', 'price': '0.00', 'avgPrice': '1633.36000', 'origQty': '6.002', 'executedQty': '6.002', 'cumQty': '6.002', 'cumQuote': '9803.42672', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': '0', 'updateTime': '1696595275687'}, 'id': '1181968724', 'clientOrderId': 'x-xcKtGhcu817b5d191973e9f64c6874', 'timestamp': 1696595275687, 'datetime': '2023-10-06T12:27:55.687Z', 'lastTradeTimestamp': 1696595275687, 'lastUpdateTimestamp': 1696595275687, 'symbol': 'ETH/USDT:USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly'

NetworkError: 1006

In [13]:
import ccxt
from datetime import datetime
from binance_api import api_key, api_secret
# Place order: {'info': {'orderId': '1181914767', 'symbol': 'ETHUSDT', 'status': 'FILLED', 'clientOrderId': 'x-xcKtGhcuc97390e896fa0ba7cc9792', 'price': '0.00', 'avgPrice': '1632.91000', 'origQty': '6.002', 'executedQty': '6.002', 'cumQty': '6.002', 'cumQuote': '9800.72582', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': '0', 'updateTime': '1696583689637'}, 'id': '1181914767', 'clientOrderId': 'x-xcKtGhcuc97390e896fa0ba7cc9792', 'timestamp': 1696583689637, 'datetime': '2023-10-06T09:14:49.637Z', 'lastTradeTimestamp': 1696583689637, 'lastUpdateTimestamp': 1696583689637, 'symbol': 'ETH/USDT:USDT', 'type': 'market', 'timeInForce': 'GTC', 'postOnly': False, 'reduceOnly': False, 'side': 'sell', 'price': 1632.91, 'triggerPrice': None, 'amount': 6.002, 'cost': 9800.72582, 'average': 1632.91, 'filled': 6.002, 'remaining': 0.0, 'status': 'closed', 'fee': {'currency': None, 'cost': None, 'rate': None}, 'trades': [], 'fees': [{'currency': None, 'cost': None, 'rate': None}], 'stopPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None}
class Whale:
    def __init__(self, exchange, symbol, leverage=100):
        self.exchange = exchange
        self.symbol = symbol
        self.leverage = leverage
        self.test_mode = True
    def get_account_balance(self):
        if self.test_mode:
            return 100
        balance = self.exchange.fetch_balance()
        return balance['total']['USDT']
    
    def get_last_price(self):
        ticker = self.exchange.fetch_ticker(self.symbol)
        return ticker['last']

    def get_order_book(self):
        order_book = self.exchange.fetch_order_book(self.symbol)
        return order_book

    def get_best_bid_and_ask(self):
        order_book = self.get_order_book()
        return order_book['bids'][0][0], order_book['asks'][0][0]

    def calculate_trade_quantity(self):
        balance = self.get_account_balance()
        leveraged_balance = min(balance * self.leverage, 100000)
        quantity = leveraged_balance / self.get_last_price()
        return quantity

    def place_trailing_stop_order(self, side):
        # Side could be 'buy' or 'sell'
        quantity = self.calculate_trade_quantity()
        best_bid, best_ask = self.get_best_bid_and_ask()
        price = best_bid if side == 'buy' else best_ask

        try:
            order = self.exchange.create_order(
                self.symbol,
                'trailing_stop',
                side,
                quantity,
                {
                    'stopPrice': price,
                    'reduceOnly': True
                }
            )
        except ccxt.BaseError as e:
            print(f"An error occurred: {e}")

        return order

    def place_first_post_only_order(self, side):
        # Side could be 'buy' or 'sell'
        quantity = self.calculate_trade_quantity()
        best_bid, best_ask = self.get_best_bid_and_ask()
        price = best_bid if side == 'buy' else best_ask

        try:
            order = self.exchange.create_limit_order(
                self.symbol,
                side,
                quantity,
                price,
                {
                    'postOnly': True,
                    'timeInForce': 'PO'  
                }
            )
        except ccxt.BaseError as e:
            print(f"An error occurred: {e}")

        return order

    def place_first_market_order(self, side):
        # Side could be 'buy' or 'sell'
        quantity = self.calculate_trade_quantity()
        order = self.exchange.create_market_order(
            self.symbol,
            side,
            quantity,
            {
                'reduceOnly': False
            }
        )
        return order

    def is_position_open(self):
        # position = self.exchange.fetch_positions([self.symbol])
        positions = self.exchange.fetch_positions([symbol])
        current_position_Amt_abs = abs(float(positions[0]['info']['positionAmt']))
        position_exists = (current_position_Amt_abs > 0 or current_position_Amt_abs < 0)
        return position_exists

    def place_second_take_profit_and_stop_loss_orders(self):
        try:
            if self.is_position_open():
                position = self.exchange.fetch_positions([self.symbol])
                if not position:
                    raise Exception("No position information found.")

                quantity = abs(float(position[0]['info']['positionAmt']))
                side = 'sell' if position[0]['side'] == 'long' else 'buy'

                entry_price = float(position[0]['info']['entryPrice'])
                take_profit_price = entry_price * (1.0007 if side == 'sell' else 0.9993)
                stop_loss_price = entry_price * (0.9993 if side == 'sell' else 1.0007)

                try:
                    take_profit_order = self.exchange.create_limit_order(
                        symbol=self.symbol,
                        side=side,
                        price=take_profit_price,
                        amount=quantity,
                        params={
                            'postOnly': True,
                            'timeInForce': 'PO',
                            'reduceOnly': True
                        }
                    )
                except ccxt.BaseError as e:
                    print(f"An error occurred in creating take-profit order: {e}")

                try:
                    stop_loss_order = self.exchange.create_order(
                        symbol=self.symbol,
                        type='stop_market',
                        side=side,
                        amount=quantity,
                        params={
                            'stopPrice': stop_loss_price,
                            'reduceOnly': True
                        }
                    )
                    
                except ccxt.BaseError as e:
                    print(f"An error occurred in creating stop-loss order: {e}")

            return take_profit_order, stop_loss_order

        except Exception as e:
            print(f"Error in placing second orders: {str(e)}")
            return None, None

    def close_open_orders(self):
        open_orders = self.exchange.fetch_open_orders(self.symbol)
        for order in open_orders:
            self.exchange.cancel_order(order['id'], self.symbol)
        return open_orders

    def close_position_with_market_order(self):
        position = self.exchange.fetch_positions([self.symbol])
        quantity = abs(float(position[0]['info']['positionAmt']))
        side = 'sell' if position[0]['side'] == 'long' else 'buy'
        order = self.exchange.create_market_order(
            self.symbol,
            side,
            quantity,
            {
                'reduceOnly': True
            }
        )
        return order

    def close_position_and_orders(self):
        try:
            self.close_open_orders()
        except Exception as e:
            print(f"Error in closing open orders: {str(e)}")

        try:
            self.close_position_with_market_order()
        except Exception as e:
            print(f"Error in closing position: {str(e)}")


# Replace with your API key and secret
symbol = 'ETH/USDT'

exchange = ccxt.binance({
    'options': {
        'defaultType': 'future',
        'newUpdates': True 
    },
    'apiKey': api_key,
    'secret': api_secret
})
# binance = exchange.load_markets()
exchange.set_sandbox_mode(True)

# Create an instance of the Whale class
whale_1 = Whale(exchange, symbol)
whale_1.calculate_trade_quantity()


6.075592522160724

In [14]:
whale_1.place_trailing_stop_order('buy')

An error occurred: binance trailing_stop is not a valid order type for the ETH/USDT market


UnboundLocalError: cannot access local variable 'order' where it is not associated with a value